In [1]:
#|default_exp net.test_graph_mermaid

In [2]:
#|export
import fbdev

In [3]:
#|export
from typing import Type, Optional, Union, Any, Tuple, Dict
import asyncio

from fbdev.comp.port import PortType, PortSpec, PortSpecCollection
from fbdev.comp import BaseComponent
from fbdev.graph.graph_spec import GraphSpec

In [4]:
class FooComponent1(BaseComponent):
    port_specs = PortSpecCollection(
        PortSpec(PortType.INPUT, "inp"),
        PortSpec(PortType.OUTPUT, "out1"),
        PortSpec(PortType.OUTPUT, "out2"),
    )
    
class FooComponent2(BaseComponent):
    port_specs = PortSpecCollection(
        PortSpec(PortType.INPUT, "inp1"),
        PortSpec(PortType.INPUT, "inp2"),
        PortSpec(PortType.OUTPUT, "out1"),
        PortSpec(PortType.OUTPUT, "out2"),
    )
    
class FooComponent3(BaseComponent):
    port_specs = PortSpecCollection(
        PortSpec(PortType.INPUT, "inp1"),
        PortSpec(PortType.INPUT, "inp2"),
        PortSpec(PortType.OUTPUT, "out"),
    )

graph = GraphSpec(PortSpecCollection())

graph.add_graph_port(PortSpec(PortType.INPUT, "inp"))
graph.add_graph_port(PortSpec(PortType.OUTPUT, "out"))

node1 = graph.add_node(FooComponent1)
node2 = graph.add_node(FooComponent2)
node3 = graph.add_node(FooComponent3)

graph.ports.input.inp >> node1.ports.input.inp
node1.ports.output.out1 >> node2.ports.input.inp1
node2.ports.input.inp2 << node1.ports.output.out2
node2.ports.output.out1 >> node3.ports.input.inp1
node2.ports.output.out2 >> node3.ports.input.inp2
node3.ports.output.out >> graph.ports.output.out

graph.display_mermaid()

```mermaid
flowchart 
    subgraph FooComponent1["FooComponent1[]"]
        FooComponent1__C__input__D__inp[inp]
        FooComponent1__C__output__D__out1[out1]
        FooComponent1__C__output__D__out2[out2]
        FooComponent1__C__message__D__started[[started]]
        FooComponent1__C__message__D__terminated[[terminated]]
    end
    subgraph FooComponent2["FooComponent2[]"]
        FooComponent2__C__input__D__inp1[inp1]
        FooComponent2__C__input__D__inp2[inp2]
        FooComponent2__C__output__D__out1[out1]
        FooComponent2__C__output__D__out2[out2]
        FooComponent2__C__message__D__started[[started]]
        FooComponent2__C__message__D__terminated[[terminated]]
    end
    subgraph FooComponent3["FooComponent3[]"]
        FooComponent3__C__input__D__inp1[inp1]
        FooComponent3__C__input__D__inp2[inp2]
        FooComponent3__C__output__D__out[out]
        FooComponent3__C__message__D__started[[started]]
        FooComponent3__C__message__D__terminated[[terminated]]
    end
    GRAPH__C__input__D__inp[inp]
    GRAPH__C__output__D__out[out]
    FooComponent2__C__output__D__out2 --> FooComponent3__C__input__D__inp2
    FooComponent1__C__output__D__out1 --> FooComponent2__C__input__D__inp1
    FooComponent1__C__output__D__out2 --> FooComponent2__C__input__D__inp2
    FooComponent3__C__output__D__out -.-> GRAPH__C__output__D__out
    GRAPH__C__input__D__inp -.-> FooComponent1__C__input__D__inp
    FooComponent2__C__output__D__out1 --> FooComponent3__C__input__D__inp1
    classDef input fill:#13543e;
    classDef output fill:#0d1b59;
    classDef subgraph_zone fill:#000;
    class FooComponent1__C__input__D__inp,FooComponent2__C__input__D__inp1,FooComponent2__C__input__D__inp2,FooComponent3__C__input__D__inp1,FooComponent3__C__input__D__inp2,GRAPH__C__input__D__inp input;
    class FooComponent1__C__output__D__out1,FooComponent1__C__output__D__out2,FooComponent2__C__output__D__out1,FooComponent2__C__output__D__out2,FooComponent3__C__output__D__out,GRAPH__C__output__D__out output;
    class FooComponent1__C__message__D__started,FooComponent1__C__message__D__terminated,FooComponent2__C__message__D__started,FooComponent2__C__message__D__terminated,FooComponent3__C__message__D__started,FooComponent3__C__message__D__terminated message;
```

In [5]:
graph.display_mermaid(hide_unconnected_ports=True)

```mermaid
flowchart 
    subgraph FooComponent1["FooComponent1[]"]
        FooComponent1__C__input__D__inp[inp]
        FooComponent1__C__output__D__out1[out1]
        FooComponent1__C__output__D__out2[out2]
    end
    subgraph FooComponent2["FooComponent2[]"]
        FooComponent2__C__input__D__inp1[inp1]
        FooComponent2__C__input__D__inp2[inp2]
        FooComponent2__C__output__D__out1[out1]
        FooComponent2__C__output__D__out2[out2]
    end
    subgraph FooComponent3["FooComponent3[]"]
        FooComponent3__C__input__D__inp1[inp1]
        FooComponent3__C__input__D__inp2[inp2]
        FooComponent3__C__output__D__out[out]
    end
    GRAPH__C__input__D__inp[inp]
    GRAPH__C__output__D__out[out]
    FooComponent2__C__output__D__out2 --> FooComponent3__C__input__D__inp2
    FooComponent1__C__output__D__out1 --> FooComponent2__C__input__D__inp1
    FooComponent1__C__output__D__out2 --> FooComponent2__C__input__D__inp2
    FooComponent3__C__output__D__out -.-> GRAPH__C__output__D__out
    GRAPH__C__input__D__inp -.-> FooComponent1__C__input__D__inp
    FooComponent2__C__output__D__out1 --> FooComponent3__C__input__D__inp1
    classDef input fill:#13543e;
    classDef output fill:#0d1b59;
    classDef subgraph_zone fill:#000;
    class FooComponent1__C__input__D__inp,FooComponent2__C__input__D__inp1,FooComponent2__C__input__D__inp2,FooComponent3__C__input__D__inp1,FooComponent3__C__input__D__inp2,GRAPH__C__input__D__inp input;
    class FooComponent1__C__output__D__out1,FooComponent1__C__output__D__out2,FooComponent2__C__output__D__out1,FooComponent2__C__output__D__out2,FooComponent3__C__output__D__out,GRAPH__C__output__D__out output;
```

In [6]:
top_graph = GraphSpec(PortSpecCollection(
    PortSpec(PortType.INPUT, "inp"),
    PortSpec(PortType.OUTPUT, "out")
))

graph_node = top_graph.add_node(fbdev.graph.GraphComponentFactory.create_component(graph))
top_graph.ports.input.inp >> graph_node.ports.input.inp
top_graph.ports.output.out << graph_node.ports.output.out

top_graph.display_mermaid()

```mermaid
flowchart 
    subgraph GraphComponent["GraphComponent[]"]
        GraphComponent__C__input__D__inp[inp]
        GraphComponent__C__output__D__out[out]
        GraphComponent__C__message__D__started[[started]]
        GraphComponent__C__message__D__terminated[[terminated]]
        subgraph GraphComponent__PROP__children[" "]
            subgraph GraphComponent__D__FooComponent1["FooComponent1[]"]
                GraphComponent__D__FooComponent1__C__input__D__inp[inp]
                GraphComponent__D__FooComponent1__C__output__D__out1[out1]
                GraphComponent__D__FooComponent1__C__output__D__out2[out2]
                GraphComponent__D__FooComponent1__C__message__D__started[[started]]
                GraphComponent__D__FooComponent1__C__message__D__terminated[[terminated]]
            end
            subgraph GraphComponent__D__FooComponent2["FooComponent2[]"]
                GraphComponent__D__FooComponent2__C__input__D__inp1[inp1]
                GraphComponent__D__FooComponent2__C__input__D__inp2[inp2]
                GraphComponent__D__FooComponent2__C__output__D__out1[out1]
                GraphComponent__D__FooComponent2__C__output__D__out2[out2]
                GraphComponent__D__FooComponent2__C__message__D__started[[started]]
                GraphComponent__D__FooComponent2__C__message__D__terminated[[terminated]]
            end
            subgraph GraphComponent__D__FooComponent3["FooComponent3[]"]
                GraphComponent__D__FooComponent3__C__input__D__inp1[inp1]
                GraphComponent__D__FooComponent3__C__input__D__inp2[inp2]
                GraphComponent__D__FooComponent3__C__output__D__out[out]
                GraphComponent__D__FooComponent3__C__message__D__started[[started]]
                GraphComponent__D__FooComponent3__C__message__D__terminated[[terminated]]
            end
        end
    end
    GRAPH__C__input__D__inp[inp]
    GRAPH__C__output__D__out[out]
    GraphComponent__D__FooComponent2__C__output__D__out2 --> GraphComponent__D__FooComponent3__C__input__D__inp2
    GraphComponent__D__FooComponent1__C__output__D__out2 --> GraphComponent__D__FooComponent2__C__input__D__inp2
    GraphComponent__D__FooComponent2__C__output__D__out1 --> GraphComponent__D__FooComponent3__C__input__D__inp1
    GRAPH__C__input__D__inp -.-> GraphComponent__C__input__D__inp
    GraphComponent__C__input__D__inp -.-> GraphComponent__D__FooComponent1__C__input__D__inp
    GraphComponent__D__FooComponent3__C__output__D__out -.-> GraphComponent__C__output__D__out
    GraphComponent__D__FooComponent1__C__output__D__out1 --> GraphComponent__D__FooComponent2__C__input__D__inp1
    GraphComponent__C__output__D__out -.-> GRAPH__C__output__D__out
    classDef input fill:#13543e;
    classDef output fill:#0d1b59;
    classDef subgraph_zone fill:#000;
    class GraphComponent__C__input__D__inp,GraphComponent__D__FooComponent1__C__input__D__inp,GraphComponent__D__FooComponent2__C__input__D__inp1,GraphComponent__D__FooComponent2__C__input__D__inp2,GraphComponent__D__FooComponent3__C__input__D__inp1,GraphComponent__D__FooComponent3__C__input__D__inp2,GRAPH__C__input__D__inp input;
    class GraphComponent__C__output__D__out,GraphComponent__D__FooComponent1__C__output__D__out1,GraphComponent__D__FooComponent1__C__output__D__out2,GraphComponent__D__FooComponent2__C__output__D__out1,GraphComponent__D__FooComponent2__C__output__D__out2,GraphComponent__D__FooComponent3__C__output__D__out,GRAPH__C__output__D__out output;
    class GraphComponent__C__message__D__started,GraphComponent__C__message__D__terminated,GraphComponent__D__FooComponent1__C__message__D__started,GraphComponent__D__FooComponent1__C__message__D__terminated,GraphComponent__D__FooComponent2__C__message__D__started,GraphComponent__D__FooComponent2__C__message__D__terminated,GraphComponent__D__FooComponent3__C__message__D__started,GraphComponent__D__FooComponent3__C__message__D__terminated message;
    class GraphComponent__PROP__children subgraph_zone;
```

In [7]:
top_graph.display_mermaid(hide_unconnected_ports=True)

```mermaid
flowchart 
    subgraph GraphComponent["GraphComponent[]"]
        GraphComponent__C__input__D__inp[inp]
        GraphComponent__C__output__D__out[out]
        subgraph GraphComponent__PROP__children[" "]
            subgraph GraphComponent__D__FooComponent1["FooComponent1[]"]
                GraphComponent__D__FooComponent1__C__input__D__inp[inp]
                GraphComponent__D__FooComponent1__C__output__D__out1[out1]
                GraphComponent__D__FooComponent1__C__output__D__out2[out2]
            end
            subgraph GraphComponent__D__FooComponent2["FooComponent2[]"]
                GraphComponent__D__FooComponent2__C__input__D__inp1[inp1]
                GraphComponent__D__FooComponent2__C__input__D__inp2[inp2]
                GraphComponent__D__FooComponent2__C__output__D__out1[out1]
                GraphComponent__D__FooComponent2__C__output__D__out2[out2]
            end
            subgraph GraphComponent__D__FooComponent3["FooComponent3[]"]
                GraphComponent__D__FooComponent3__C__input__D__inp1[inp1]
                GraphComponent__D__FooComponent3__C__input__D__inp2[inp2]
                GraphComponent__D__FooComponent3__C__output__D__out[out]
            end
        end
    end
    GRAPH__C__input__D__inp[inp]
    GRAPH__C__output__D__out[out]
    GraphComponent__D__FooComponent2__C__output__D__out2 --> GraphComponent__D__FooComponent3__C__input__D__inp2
    GraphComponent__D__FooComponent1__C__output__D__out2 --> GraphComponent__D__FooComponent2__C__input__D__inp2
    GraphComponent__D__FooComponent2__C__output__D__out1 --> GraphComponent__D__FooComponent3__C__input__D__inp1
    GRAPH__C__input__D__inp -.-> GraphComponent__C__input__D__inp
    GraphComponent__C__input__D__inp -.-> GraphComponent__D__FooComponent1__C__input__D__inp
    GraphComponent__D__FooComponent3__C__output__D__out -.-> GraphComponent__C__output__D__out
    GraphComponent__D__FooComponent1__C__output__D__out1 --> GraphComponent__D__FooComponent2__C__input__D__inp1
    GraphComponent__C__output__D__out -.-> GRAPH__C__output__D__out
    classDef input fill:#13543e;
    classDef output fill:#0d1b59;
    classDef subgraph_zone fill:#000;
    class GraphComponent__C__input__D__inp,GraphComponent__D__FooComponent1__C__input__D__inp,GraphComponent__D__FooComponent2__C__input__D__inp1,GraphComponent__D__FooComponent2__C__input__D__inp2,GraphComponent__D__FooComponent3__C__input__D__inp1,GraphComponent__D__FooComponent3__C__input__D__inp2,GRAPH__C__input__D__inp input;
    class GraphComponent__C__output__D__out,GraphComponent__D__FooComponent1__C__output__D__out1,GraphComponent__D__FooComponent1__C__output__D__out2,GraphComponent__D__FooComponent2__C__output__D__out1,GraphComponent__D__FooComponent2__C__output__D__out2,GraphComponent__D__FooComponent3__C__output__D__out,GRAPH__C__output__D__out output;
    class GraphComponent__PROP__children subgraph_zone;
```